In [21]:
import numpy as np
import pandas as pd
import datetime

In [8]:
df = pd.read_csv('music_data.csv', sep=',')

In [15]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [9]:
# Приведем даты к нормальному формату

df['utc_audition_start_dttm'] = df['utc_audition_start_dttm'].str.split('Z').str.get(0)
df['utc_audition_start_dttm'] = pd.to_datetime(df['utc_audition_start_dttm'])
df['utc_audition_end_dttm'] = df['utc_audition_end_dttm'].str.split('Z').str.get(0)
df['utc_audition_end_dttm'] = pd.to_datetime(df['utc_audition_end_dttm'])

In [112]:
# Посмотрим пользователей

df['user_id'].value_counts()

0041383820e248e487a6    1069
34a0b3746c624e5ba3ca    1022
26fc025750364a34881f    1011
805f5e31be634900a9ed     923
73949f6f48b746e0bbd4     890
                        ... 
76e38ef9ef7344c78863     115
e787dd56be6b4defa646     111
e4e38371f16149f6a10c     107
1828b1036eaf4e64841a      72
52954001ebf6474586ba      68
Name: user_id, Length: 1727, dtype: int64

In [116]:
# Посчитаем сколько секунд пользователь слушал каждый трек

df["listen_duration"] = df["utc_audition_end_dttm"] - df["utc_audition_start_dttm"]

df["listen_duration"] = (
    pd.to_timedelta(df["listen_duration"]).dt.total_seconds().astype(float)
)

In [27]:
# Посчитаем долю прослушанного времени от длины всей песни

df['listen_pctg'] = df['listen_duration'] / df['track_duration']

In [86]:
# Создадим вспомогательный датафреймы, чтобы посчитать время и количество треков

# Время всего прослушенного каждым пользователем 
df_a = df.copy()
df_ad = df_a.groupby('user_id', as_index=False)['listen_duration'].sum()
df_ad.columns = ['user_id', 'listen_duration_all']

# Кол-во треков всего прослушенных каждым пользователем 
df_at = df_a.groupby('user_id', as_index=False)['listen_duration'].count()
df_at.columns = ['user_id', 'count_track_all']

# Время треков Techno прослушенных каждым пользователем 
df_t = df.copy()
df_t = df_t[df_t['track_genre'] == 'TECHNO']
df_td = df_t.groupby('user_id', as_index=False)['listen_duration'].sum()
df_td.columns = ['user_id', 'listen_duration_techno']

# Кол-во треков Techno прослушенных каждым пользователем
df_tt = df_t.groupby('user_id', as_index=False)['listen_duration'].count()
df_tt.columns = ['user_id', 'count_track_techno']

# Время треков жанров, похожих на Techno, прослушенных каждым пользователем
df_e = df.copy()
df_e = df_e[
            (df_e['track_genre'] == 'DUBSTEP') | 
            (df_e['track_genre'] == 'ELECTRONICS') | 
            (df_e['track_genre'] == 'HOUSE')
            ]
df_ed = df_e.groupby('user_id', as_index=False)['listen_duration'].sum()
df_ed.columns = ['user_id', 'listen_duration_e']

# Кол-во треков жанров, похожих на Techno, прослушенных каждым пользователем
df_et = df_e.groupby('user_id', as_index=False)['listen_duration'].count()
df_et.columns = ['user_id', 'count_track_e']

In [119]:
# Создадим таблицу с аналитикой по времени

df_md = df_ad.merge(df_td, on="user_id")
df_md["listen_pct_t"] = df_md["listen_duration_techno"] / df_md["listen_duration_all"]

df_md = df_md.merge(df_ed, on="user_id")
df_md["listen_pct_e"] = df_md["listen_duration_e"] / df_md["listen_duration_all"]
df_md["listen_pct_e+t"] = (
    df_md["listen_duration_techno"] + df_md["listen_duration_e"]
) / df_md["listen_duration_all"]


In [111]:
# Создадим таблицу с аналитикой по количеству треков

df_mt = df_at.merge(df_tt, on="user_id")
df_mt["track_pct_t"] = df_mt["count_track_techno"] / df_mt["count_track_all"]

df_mt = df_mt.merge(df_et, on="user_id")
df_mt["track_pct_e"] = df_mt["count_track_e"] / df_mt["count_track_all"]
df_mt["track_pct_e+t"] = (df_mt["count_track_techno"] + df_mt["count_track_e"]) / df_mt[
    "count_track_all"
]


In [109]:
# Создадим единую таблицу и сделаем отбор по параметрам, чтобы выявить 
# потенциальных участников мероприятия

df_merge = df_md.merge(df_mt, on='user_id')
df_merge[(df_merge['listen_pct_e+t']>0.17) & 
         (df_merge['track_pct_e+t']>0.17) &
         (df_merge['listen_pct_t']>0.05) & 
         (df_merge['track_pct_t']>0.05)]

,user_id,listen_duration_all,listen_duration_techno,listen_pct_t,listen_duration_e,listen_pct_e,listen_pct_e+t,count_track_all,count_track_techno,track_pct_t,count_track_e,track_pct_e,track_pct_e+t
3,00c017dc34254d39a390,273406.105,17279.222,0.063200,35123.674,0.128467,0.191667,633,51,0.080569,90,0.142180,0.222749
9,01936de0105345d38eb4,295464.567,17075.847,0.057793,36442.005,0.123338,0.181131,695,54,0.077698,95,0.136691,0.214388
24,03dc15d2a66144368e84,125638.183,7594.580,0.060448,15046.996,0.119765,0.180213,341,22,0.064516,38,0.111437,0.175953
25,03f59562ca5c428a9425,148451.533,9691.699,0.065285,17102.728,0.115207,0.180493,340,29,0.085294,44,0.129412,0.214706
26,04564d08b879495d9e40,117455.873,6714.793,0.057169,14853.290,0.126458,0.183627,359,25,0.069638,44,0.122563,0.192201
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,fbc74bb744cc46b3a401,160493.480,12248.679,0.076319,18282.111,0.113912,0.190231,424,33,0.077830,52,0.122642,0.200472
1709,fcee3c179fc641aab75d,146504.477,8998.920,0.061424,19134.595,0.130608,0.192032,333,25,0.075075,44,0.132132,0.207207
1713,fd7a520bcda845caa558,338090.056,19614.084,0.058014,47580.792,0.140734,0.198748,822,59,0.071776,122,0.148418,0.220195
1722,ff77d34a23c04d36a421,168070.920,9823.367,0.058448,21514.728,0.128010,0.186458,402,33,0.082090,56,0.139303,0.221393
